In [269]:
import ClusterMangager
import Query
import Connector
import main

import pandas as pd
import numpy as np
from elasticsearch import Elasticsearch
from importlib import reload

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [270]:
# Uncomment for reloading
reload(Query)
reload(main)
reload(ClusterMangager)

imported query handler
imported elastic pipeline
imported indexing library


<module 'ClusterMangager' from 'c:\\Users\\DangNNH2\\Documents\\vscode\\demo-elastic-search\\ClusterMangager.py'>

In [271]:
%time
index_name = 'test-es-indexing'
data_file = 'data/corpus.csv'
query_file = 'data/test_query.csv'

print("loading data...")
sample_cols = ['full', 'doc_id']
query_cols = ['question', 'doc_id']
data_df = main.load_data(data_file = data_file, usecols=sample_cols)
query_df = main.load_data(data_file = query_file, usecols=query_cols)
# data_df = data_df.iloc[:100]
print("done loading\n")

client = Connector.connect2es()
print("connected to elastic client\n")

print("Sample data")

CPU times: total: 0 ns
Wall time: 0 ns
loading data...
done loading

connected to elastic client

Sample data


C:\Users\DangNNH2\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\_sync\client\__init__.py:375: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [272]:
data_df

,full,doc_id
0,Điều 1. Phạm vi áp dụng Thông tư này hướng dẫn...,0
1,Điều 2. Tổ chức lực lượng 1. Hàng năm trước mù...,1
2,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,2
3,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",3
4,"Điều 5. Phù hiệu của lực lượng tuần tra, canh ...",4
...,...,...
61420,Điều 60. Thẩm quyền xử phạt của Hải quan 1. Ch...,61420
61421,Điều 61. Thẩm quyền xử phạt của Quản lý thị tr...,61421
61422,Điều 62. Thẩm quyền xử phạt của Thanh tra 1. T...,61422
61423,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,61423


In [273]:
indices = ClusterMangager.get_all_index(client)
for index_name in indices:
    try:
        
        print("delete old '%s'" % index_name)
        client.indices.delete(index=index_name)
        client.indices.exists(index=index_name)
    except Exception:
        print("index '%s' created" % index_name)
    finally:
        print("index '%s' deleted" % index_name)

delete old 'test-es-indexing'
index 'test-es-indexing' deleted


In [274]:
print("indexing data...")
ClusterMangager.indexing_source(index_name, data_df, client, _id_='doc_id')
print("done indexing\n")

indexing data...
indexing at 0
indexing at 100
indexing at 200
indexing at 300
indexing at 400
indexing at 500
indexing at 600
indexing at 700
indexing at 800
indexing at 900
indexing at 1000
indexing at 1100
indexing at 1200
indexing at 1300
indexing at 1400
indexing at 1500
indexing at 1600
indexing at 1700
indexing at 1800
indexing at 1900
indexing at 2000
indexing at 2100
indexing at 2200
indexing at 2300
indexing at 2400
indexing at 2500
indexing at 2600
indexing at 2700
indexing at 2800
indexing at 2900
indexing at 3000
indexing at 3100
indexing at 3200
indexing at 3300
indexing at 3400
indexing at 3500
indexing at 3600
indexing at 3700
indexing at 3800
indexing at 3900
indexing at 4000
indexing at 4100
indexing at 4200
indexing at 4300
indexing at 4400
indexing at 4500
indexing at 4600
indexing at 4700
indexing at 4800
indexing at 4900
indexing at 5000
indexing at 5100
indexing at 5200
indexing at 5300
indexing at 5400
indexing at 5500
indexing at 5600
indexing at 5700
indexing 

In [275]:
data_df

,full,doc_id
0,Điều 1. Phạm vi áp dụng Thông tư này hướng dẫn...,0
1,Điều 2. Tổ chức lực lượng 1. Hàng năm trước mù...,1
2,Điều 3. Tiêu chuẩn của các thành viên thuộc lự...,2
3,"Điều 4. Nhiệm vụ của lực lượng tuần tra, canh ...",3
4,"Điều 5. Phù hiệu của lực lượng tuần tra, canh ...",4
...,...,...
61420,Điều 60. Thẩm quyền xử phạt của Hải quan 1. Ch...,61420
61421,Điều 61. Thẩm quyền xử phạt của Quản lý thị tr...,61421
61422,Điều 62. Thẩm quyền xử phạt của Thanh tra 1. T...,61422
61423,Điều 63. Phân định thẩm quyền xử phạt của Chủ ...,61423


In [276]:
# test = data_df.iloc[309]
# doc = test['text'].to_dict()
# client.index(index=index_name, id=309, document=data_df)
# test

In [277]:
sample_query = query_df.iloc[1].question
print("Retrive documents for sample query: ", sample_query)
resp = Query.query4phrase(client, index=index_name, question=sample_query, query_on='full', return_field=['_id'], top_k=10)
dict(resp['hits'])

Retrive documents for sample query:  Phải thực hiện thao tác nạp mẫu vào bình chứa và xử lý mẫu sơ bộ bằng hóa chất như thế nào?


{'total': {'value': 10000, 'relation': 'gte'},
 'max_score': 61.510326,
 'hits': [{'_index': 'test-es-indexing',
   '_id': '43168',
   '_score': 61.510326,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '43165',
   '_score': 43.24275,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '4367',
   '_score': 39.173813,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '43169',
   '_score': 36.141716,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '29410',
   '_score': 34.634823,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '47708',
   '_score': 34.012135,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '4362',
   '_score': 33.29708,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '13895',
   '_score': 32.590412,
   '_ignored': ['full.keyword']},
  {'_index': 'test-es-indexing',
   '_id': '

In [278]:
# es.indices.delete(index =['just-test'])
# es.indices.exists(index= ['just-test'])

In [279]:
top_k = 10
predictions = Query.predict_all_revelants_doc(client, index='test-es-indexing', query_on='full', question_df=query_df.question)

In [280]:
predictions = predictions.astype(int)
print(predictions)
print(predictions.shape)

[[10319 10335 10330 ... 10322 40204 10360]
 [43168 43165  4367 ... 13895  4333 44376]
 [22736 36658 35226 ... 59261 48566   264]
 ...
 [47997 42989 44649 ... 47977 41172 42963]
 [10318 39952 13129 ... 18162  1826 19606]
 [38338 38333 38332 ... 38305 18452 59843]]
(3298, 10)


In [281]:
labels = query_df.doc_id.values
print(labels)
print(labels.shape)

[45558 43168 22736 ... 42952 10318 38333]
(3298,)


In [282]:
# reload(main)
# print(len(labels))
print("At k =", top_k, 'the perfomance of elastic is: ')
score = main.gen_report(predictions, labels)

At k = 10 the perfomance of elastic is: 
Precision            ||   0.8071558520315343
Mean Reciporal Rank  ||   0.7686335874556007


# Report note:

- The prototype is done on local machine with 16gb RAM an Intel 10th U-chip
- Time complexity:
    - **Indexing time** for corpus of **64k document** is around **3 minute**
    - **Query time** for **3k queries** is around **30s**
- Performance is overly **outperforming** custom methods :((
    - Retriving only top 10 document with highest score return
    <pre>
        - Precison:            0.8071558520315343 (1)
        - Mean Reciporal Rank: 0.7686335874556007 (1)
    </pre>
- Elasticsearch does not support built-in natural language processing pipeline for Vietnamese
    - Can a custom pipeline improve the performance in term of relevancy results ?
